<a href="https://colab.research.google.com/github/iamkaiwei/3min-Android/blob/master/boringPpl_similarity_match.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Standard Imports

In [25]:
NULL_COMPANY_RANK = 300000

In [1]:
# system imports
import sys
import re
import json
import string

# 3rd party imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# config
plt.style.use('ggplot')
np.set_printoptions(threshold=sys.maxsize)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Import Google Drive Auth

In [2]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

# Import Data from Google Sheets Link

In [3]:
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1RIS29R2HkVxxonr8DCTgfttZtcLraw1mj5dwn06_SMk/edit?pli=1#gid=1743435448')

In [4]:
sheet = wb.worksheet('intent')
data = sheet.get_all_values()

In [5]:
df_1 = pd.DataFrame(data)       # convert object from list to df
df_1.columns = df_1.iloc[0,:]   # convert first row to headers
df_1 = df_1.iloc[1:]            # trim first row

In [6]:
# Base cleaning

df_1['experience_clean'] = df_1['experience'].apply(lambda x: json.loads(x))
df_1['education_clean'] = df_1['education'].apply(lambda x: json.loads(x))
df_1['_id'] = df_1.index

In [7]:
df_1.head()

,Domain,intent,subtitle,experience,education,experience_clean,education_clean,_id
1,Software Development,Keep the coding momentum,Intern at Enspire Capital,"[{""roles"": [{""name"": ""Intern"", ""duration"": ""Ju...","[{""school"": ""Singapore University of Technolog...","[{'roles': [{'name': 'Intern', 'duration': 'Ju...",[{'school': 'Singapore University of Technolog...,1
2,Natural Language Processing,Learning and exploring powerful python librari...,Passionate about data science | Ex-SIA,"[{""roles"": [{""name"": ""Data Analyst"", ""duration...","[{""school"": ""Singapore Management University"",...","[{'roles': [{'name': 'Data Analyst', 'duration...","[{'school': 'Singapore Management University',...",2
3,Data Analytics,Targeted goal: be able to conduct data analyse...,Aspiring Product Manager | Data Scientist | Pr...,"[{""roles"": [{""name"": ""Quantitative Analyst, Ma...","[{""school"": ""University of Cambridge"", ""degree...","[{'roles': [{'name': 'Quantitative Analyst, Ma...","[{'school': 'University of Cambridge', 'degree...",3
4,Data Analytics,I want to learn about machine learning and dat...,Regional Business Analyst Intern at Ninja Van,"[{""roles"": [{""name"": ""Regional Business Analys...","[{""school"": ""National University of Singapore""...",[{'roles': [{'name': 'Regional Business Analys...,[{'school': 'National University of Singapore'...,4
5,Python programming,"Learn the basic functions of python, learn how...",Data Analyst Intern at DBS Bank,"[{""roles"": [{""name"": ""Data Analyst Intern"", ""d...","[{""school"": ""Singapore Management University"",...","[{'roles': [{'name': 'Data Analyst Intern', 'd...","[{'school': 'Singapore Management University',...",5


In [8]:
df_1.iloc[0]['experience']

'[{"roles": [{"name": "Intern", "duration": "Jul 2020 ‚Äì Present", "location": "Singapore", "description": ""}], "company": "Enspire Capital", "company_crunchbase_slug": "enspire-capital", "company_cb_rank": "12,296"}, {"roles": [{"name": "Summer Intern", "duration": "Jun 2020 ‚Äì Present", "location": "", "description": ""}], "company": "BANSEA", "company_crunchbase_slug": "bansea", "company_cb_rank": "13,447"}, {"roles": [{"name": "Associate", "duration": "Jun 2020 ‚Äì Present", "location": "", "description": ""}, {"name": "Investment Analyst", "duration": "Jan 2020 ‚Äì Jun 2020", "location": "", "description": ""}, {"name": "Analyst in Training", "duration": "Aug 2019 ‚Äì Jan 2020", "location": "", "description": ""}], "company": "Prot√©g√© Ventures", "company_crunchbase_slug": "prot√©g√©-ventures", "company_cb_rank": "36,589"}, {"roles": [{"name": "President", "duration": "Nov 2019 ‚Äì Present", "location": "Singapore", "description": "Organising events, spearheading new projects,

In [26]:
import json
from datetime import datetime


def clean_linkedin_date(dt):
  """
  Converts linkedin date to pd.Timestamp

  dt: str, a string date 
  """
  try:
    return pd.Timestamp(dt)
  except:
    if 'Present' in dt:
      # TODO: should be last updated linkedin data feed
      return pd.Timestamp(datetime.now())
    else:
      raise Exception('Unknown date format found in `duration`')


def clean_and_expand_role(role):
  """
  Parses out the role object from linkein

  role: dict, linkedin role object

  Note: created features prepended with _
  """
  out = role.copy()
  out['duration_clean'] = out['duration'].encode('ascii', 'ignore').decode()
  out['_start_date'] = clean_linkedin_date(out['duration_clean'].split('  ')[0])
  try:
    out['_end_date'] = clean_linkedin_date(out['duration_clean'].split('  ')[1])
  
  except IndexError:
    out['_end_date'] = out['_start_date']

  out['_duration_tdelta'] = out['_end_date'] - out['_start_date']
  return out


def parse_experience(experience_id, experience, take_latest_roles=3):
  """
  Creates the feature columns from raw linkedin experiences

  experience_id: int, uid for experience
  experience: dict, linkedin experience object
  take_latest_roles: int, specify number of latest roles to extract from
  """
  if not experience:
    return [{}, {}, {}]

  try:
    latest_roles = []
    for company_idx, company in enumerate(experience):
      # TODO: remove if linkedin guarentees chron order of roles
      date_sorted_roles = sorted(
          company['roles'], 
          key=lambda x: clean_linkedin_date(
              dt=clean_and_expand_role(role=x)['_end_date']
          ),
          reverse=True)
      
      for role in date_sorted_roles:
        clean_role = clean_and_expand_role(role=role)
        clean_role['role_id'] = len(latest_roles)
        clean_role['company_id'] = company_idx
        clean_role['company_name'] = company['company']
        try: 
          clean_role['company_cb_rank'] = company['company_cb_rank']
        except KeyError as e:
          clean_role['company_cb_rank'] = NULL_COMPANY_RANK
        clean_role = {
            '{}th_role-{}'.format(len(latest_roles), k): v
            for k,v in clean_role.items()
        }
        if len(latest_roles) < take_latest_roles:
          latest_roles.append(clean_role)
        else:
          break
      
    out = []
    for i in range(take_latest_roles):
      try:
        out.append(latest_roles[i])
      except:
        if len(out) == 0:
          raise Exception(
              'This individual does not have {} roles.'
              .format(take_latest_roles)
          )
        
        out.append({k: np.nan for k in out[0]})

    return out

  except Exception as e:
    print('Problem experience: ', experience_id)
    print('Problem company: ', company)
    print('Problem role: ', role)
    print('Problem role: ', role)
    raise(e)

See `df_3` is derived from `df_1`, but only contains the parsed experience column attributes.

In [27]:
df_2 = df_1.apply(axis=1, 
           func=lambda x: parse_experience(experience_id=x['_id'],
                                           experience=x['experience_clean']),
           result_type='expand')
df_2.columns = [str(item) for item in df_2.columns]

# TODO: find the bug causing dropna here; 0th getting added to all
dfs = [
       pd.DataFrame(df_2[col].values.tolist()).dropna(axis=1, how='all') 
       for col in df_2.columns
      ]

df_3 = pd.DataFrame([])
for df in dfs:
  if df_3.empty:
    df_3 = df
  else:
    df_3 = df_3.join(other=df)

In [28]:
df_1.iloc[1].experience_clean

[{'company': 'Juwwa Inc',
  'roles': [{'description': '',
    'duration': 'Jan 2020 ‚Äì Present',
    'location': 'Tokyo,Japan',
    'name': 'Data Analyst'}]},
 {'company': 'Singapore Airlines',
  'company_cb_rank': '48,179',
  'company_crunchbase_slug': 'singapore-airlines',
  'roles': [{'description': '',
    'duration': 'Mar 2018 ‚Äì Oct 2019',
    'location': 'Singapore',
    'name': 'Business Analyst - Revenue Generation and Customer Experience'}]},
 {'company': 'TrustSphere',
  'company_cb_rank': '91,371',
  'company_crunchbase_slug': 'trustsphere',
  'roles': [{'description': '',
    'duration': 'Apr 2017 ‚Äì Feb 2018',
    'location': 'Singapore',
    'name': 'Sales and People Analyst'}]},
 {'company': 'Mahindra Comviva',
  'company_cb_rank': '124,949',
  'company_crunchbase_slug': 'mahindra-comviva',
  'roles': [{'description': '',
    'duration': 'Nov 2014 ‚Äì Jul 2016',
    'location': 'Gurgaon, India',
    'name': 'Software Engineer'}]},
 {'company': 'CNN-IBN',
  'roles': [

In [29]:
df_3.dtypes

0th_role-name                         object
0th_role-duration                     object
0th_role-location                     object
0th_role-description                  object
0th_role-duration_clean               object
0th_role-_start_date          datetime64[ns]
0th_role-_end_date            datetime64[ns]
0th_role-_duration_tdelta    timedelta64[ns]
0th_role-role_id                     float64
0th_role-company_id                  float64
0th_role-company_name                 object
0th_role-company_cb_rank              object
1th_role-name                         object
1th_role-duration                     object
1th_role-location                     object
1th_role-description                  object
1th_role-duration_clean               object
1th_role-_start_date          datetime64[ns]
1th_role-_end_date            datetime64[ns]
1th_role-_duration_tdelta    timedelta64[ns]
1th_role-role_id                     float64
1th_role-company_id                  float64
1th_role-c

Looking at the most recent role

In [30]:
most_recent_role = [col for col in df_3.columns if '0th' in col]
one_hot_columns = ['0th_role-location', '0th_role-company_name']

In [31]:
df_3[most_recent_role].head()

,0th_role-name,0th_role-duration,0th_role-location,0th_role-description,0th_role-duration_clean,0th_role-_start_date,0th_role-_end_date,0th_role-_duration_tdelta,0th_role-role_id,0th_role-company_id,0th_role-company_name,0th_role-company_cb_rank
0,Intern,Jul 2020 ‚Äì Present,Singapore,,Jul 2020 Present,2020-07-01,2020-08-19 12:42:01.956788,49 days 12:42:01.956788,0.0,0.0,Enspire Capital,"12,296"
1,Data Analyst,Jan 2020 ‚Äì Present,"Tokyo,Japan",,Jan 2020 Present,2020-01-01,2020-08-19 12:42:01.959239,231 days 12:42:01.959239,0.0,0.0,Juwwa Inc,300000
2,"Quantitative Analyst, Marketing",Sep 2019 ‚Äì Present,,I work on Google Search & Assistant Marketing ...,Sep 2019 Present,2019-09-01,2020-08-19 12:42:01.960634,353 days 12:42:01.960634,0.0,0.0,Google,276
3,Regional Business Analyst Intern,Apr 2020 ‚Äì Present,Singapore,,Apr 2020 Present,2020-04-01,2020-08-19 12:42:01.961771,140 days 12:42:01.961771,0.0,0.0,Ninja Van,"1,953"
4,Data Analyst Intern,May 2020 ‚Äì Present,,,May 2020 Present,2020-05-01,2020-08-19 12:42:01.963131,110 days 12:42:01.963131,0.0,0.0,DBS Bank,"42,439"


Finding similarity between role names on whole words.

See `ans2` for a summary of how many people's current role contains each keyword.

In [32]:
def strip_word(word):
  if pd.isnull(word) or word == '':
    return ''

  tmp = word
  for num in '0123456789':
    if word == num:
      return ''

  for punc in string.punctuation:
    # TODO: account for double parens in front
    yes = True
    while yes:
      before = tmp
      tmp = tmp.replace(punc, '')
      if tmp == before:
        yes = False

  tmp = tmp.replace(' ', '')
  return tmp


def nan_in_wrapper(a, b, case_sensitive=False):
  """
  checks if a in b
  """
  if pd.isnull(a) or pd.isnull(b):
    return False
  
  if case_sensitive: 
    return a in b
  else:  
    return a.lower() in b.lower()


unique_words_rolename = set()
role_name_split_len = []

for row in df_3[most_recent_role].to_dict('records'):
  if pd.isnull(row['0th_role-name']):
    continue
  
  splt = row['0th_role-name'].split(' ')
  role_name_split_len.append(len(splt))

  for word in splt:
    stripped_word = strip_word(word=word)
    if not stripped_word:
      continue
    
    unique_words_rolename.add(stripped_word.lower())

unique_words_rolename = sorted(list(unique_words_rolename))

print('{} unique words in {} roles ({} words total)'.format(
    len(unique_words_rolename), 
    len(df_3),
    sum(role_name_split_len)
))
print('Average word count per role-name: {}'.format(np.average(role_name_split_len)))
print('')

ans = []
role_name_col = [col for col in df_3.columns if 'role-name' in col][0]
for word in unique_words_rolename:
  ans.append(df_3[role_name_col].apply(
      lambda x: nan_in_wrapper(word, strip_word(x)))
  )

ans2 = pd.concat(ans, axis=1)
ans2.columns = unique_words_rolename
print('Total people with current role name that includes keyword')
ans2.sum().sort_values(ascending=False)

79 unique words in 76 roles (178 words total)
Average word count per role-name: 2.870967741935484

Total people with current role name that includes keyword


data            21
engineer        16
analyst         12
intern          11
scientist       11
                ..
junior           1
intelligence     1
information      1
human            1
intel¬æ          1
Length: 79, dtype: int64

Coincidences looks at the columns of `df_3` (the parsed experiences) that were easily one-hot encoded

In [33]:
df_2.head().iloc[0][1]

{'1th_role-_duration_tdelta': Timedelta('79 days 12:42:01.957063'),
 '1th_role-_end_date': Timestamp('2020-08-19 12:42:01.957063'),
 '1th_role-_start_date': Timestamp('2020-06-01 00:00:00'),
 '1th_role-company_cb_rank': '13,447',
 '1th_role-company_id': 1,
 '1th_role-company_name': 'BANSEA',
 '1th_role-description': '',
 '1th_role-duration': 'Jun 2020 ‚Äì Present',
 '1th_role-duration_clean': 'Jun 2020  Present',
 '1th_role-location': '',
 '1th_role-name': 'Summer Intern',
 '1th_role-role_id': 1}

In [34]:
# Looking at coincidences in location or company

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()
one_hot_features = df_3[one_hot_columns]
one_hot_features.fillna(value='', inplace=True)
onehots = enc.fit_transform(one_hot_features).toarray()

# Number of people with something in common
coincidences = pd.DataFrame(
    data=onehots,
    columns=enc.get_feature_names(input_features=one_hot_columns)
  ).sum().sort_values(ascending=False)

# printing summary of categories with > 1 person
coincidences[(coincidences  > 1)]

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


0th_role-location_                                     28.0
0th_role-company_name_                                 14.0
0th_role-location_Singapore                            14.0
0th_role-company_name_Bank of America Merrill Lynch     2.0
0th_role-company_name_OMNO AI                           2.0
0th_role-location_Mumbai Area, India                    2.0
0th_role-location_Bengaluru, Karnataka                  2.0
dtype: float64

In [41]:
df_3.columns

Index(['0th_role-name', '0th_role-duration', '0th_role-location',
       '0th_role-description', '0th_role-duration_clean',
       '0th_role-_start_date', '0th_role-_end_date',
       '0th_role-_duration_tdelta', '0th_role-role_id', '0th_role-company_id',
       '0th_role-company_name', '0th_role-company_cb_rank', '1th_role-name',
       '1th_role-duration', '1th_role-location', '1th_role-description',
       '1th_role-duration_clean', '1th_role-_start_date', '1th_role-_end_date',
       '1th_role-_duration_tdelta', '1th_role-role_id', '1th_role-company_id',
       '1th_role-company_name', '1th_role-company_cb_rank', '2th_role-name',
       '2th_role-duration', '2th_role-location', '2th_role-description',
       '2th_role-duration_clean', '2th_role-_start_date', '2th_role-_end_date',
       '2th_role-_duration_tdelta', '2th_role-role_id', '2th_role-company_id',
       '2th_role-company_name', '2th_role-company_cb_rank'],
      dtype='object')

In [44]:
df_3[pd.isnull(df_3["0th_role-company_name"])].dropna(how="all")

,0th_role-name,0th_role-duration,0th_role-location,0th_role-description,0th_role-duration_clean,0th_role-_start_date,0th_role-_end_date,0th_role-_duration_tdelta,0th_role-role_id,0th_role-company_id,0th_role-company_name,0th_role-company_cb_rank,1th_role-name,1th_role-duration,1th_role-location,1th_role-description,1th_role-duration_clean,1th_role-_start_date,1th_role-_end_date,1th_role-_duration_tdelta,1th_role-role_id,1th_role-company_id,1th_role-company_name,1th_role-company_cb_rank,2th_role-name,2th_role-duration,2th_role-location,2th_role-description,2th_role-duration_clean,2th_role-_start_date,2th_role-_end_date,2th_role-_duration_tdelta,2th_role-role_id,2th_role-company_id,2th_role-company_name,2th_role-company_cb_rank


The above 3rd line reads, 14 people's most recent role based in Singapore.

TODO: Remove the first 2 keys from this list: `0th_role-location_` and `0th_role-company_name_` which are roles that have no information for location or company_name.  Second thought, this is strange that a role would not have a company name.  Look into that.

# Pre-process Data part 1, field: education


1. Goal of this section is to flatten each column of nested fields into multiple columns

2. A new row is only generated when there is a) a new individual b) a new survey response from an existing individual (what if an existing individual changes some details on his LinkedIn profile or changes his/her goals?)

In [35]:
df_1.describe()

,_id
count,76.000000
mean,38.500000
std,22.083176
min,1.000000
25%,19.750000
50%,38.500000
75%,57.250000
max,76.000000


In [45]:
# count the number of words in all rows
# remove punctuation and split string into individual words

def word_counter(i):
  
  # i is an array of rows for a particular field
  word_count = dict()

  for x in range(1,i.shape[0]):
    j = df_1.loc[i,['education']][0].translate(str.maketrans('', '', string.punctuation)).split()
  for k in j:
    if k in word_count:
      word_count[k] += 1
    else:
      word_count[k] = 1
    return word_count

In [46]:
schools_list_1 = []

for i in range(1,df_1.shape[0]):
  
  # Check if there is a first school. If yes, extract the first school into a list to create a new column, else indicate 'no school'
  if df_1.loc[i,['education']][0].find('"school": "') == -1:
    school_1 = 'no school'
  else:
    school_1 = (df_1.loc[i,['education']][0][
      df_1.loc[i,['education']][0].find('"school": "')+10:
      df_1.loc[i,['education']][0].find('", "degree":')                         
      ])
  schools_list_1.append(school_1)

  # Check if there is a second school. If yes, extract the 2nd school into a list 'schools_list_2' for a subsequent column, else indicate 'n/a'


# convert all 
print(pd.DataFrame(schools_list_1))

                                                    0
0   "Singapore University of Technology and Design...
1                    "Singapore Management University
2                            "University of Cambridge
3                   "National University of Singapore
4                    "Singapore Management University
..                                                ...
70                   "Singhad Institute of Technology
71       "Sambhram Institute of Technology, BANGALORE
72                                          no school
73        "African Institute of Mathematical Sciences
74  "National University of Computer and Emerging ...

[75 rows x 1 columns]


## Next steps: 

- Clean up bug where companies dont exist. @cyan @qinglin
- Add in crunchbase ID, if not substitute with another value. ie: 300k @kai
- 